<a href="https://colab.research.google.com/github/Deriverx2/LinearAlgebraQuestions/blob/master/Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-0311b558-26ad-592a-e064-19fdfcb3ece1)


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloom-1b1",
    load_in_8bit = True,
    device_map = "auto"
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.13G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Freezing original weights

In [ ]:
for param in model.parameters():
  param.requires_grad = False
  if param.ndim == 1:
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self,x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)


Setting Up LORA Adapters

In [ ]:
def print_trainable_parameters(model):
  """
  Print number of trainable parameters in the model
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(f"trainable_params: {trainable_params} || all_params: {all_param} || trainable%: {100 * trainable_params/all_param} ")

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    # lora_droupout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

print_trainable_parameters(model)

trainable_params: 2359296 || all_params: 1067673600 || trainable%: 0.22097539922313336 


In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset ("Likhi2003/linearalgebra_QA")

README.md:   0%|          | 0.00/269 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/59.2k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/223 [00:00<?, ? examples/s]

In [ ]:
data['train'][0]["text"]

'<s>[INST] Level-1: What is the matrix-vector multiplication of a 2x2 matrix A = [a11, a12] and a 1x2 vector x = [x1]? [/INST] Ax = [a11x1, a12x1] </s>'

In [ ]:
data = data.map(lambda samples: tokenizer(samples['text']), batched=True)
# data

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 223
    })
})

Training

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
    ),
    # removed_unused_columns=True,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: deriverx2 (deriverx2-cochin-university-of-science-and-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,1.653800
2,1.810000
3,1.890000
4,1.864400
5,1.854500
6,1.687200
7,1.622200
8,1.711000
9,1.693200
10,1.799100


TrainOutput(global_step=200, training_loss=1.7798353880643845, metrics={'train_runtime': 855.2945, 'train_samples_per_second': 3.741, 'train_steps_per_second': 0.234, 'total_flos': 3773446278819840.0, 'train_loss': 1.7798353880643845, 'epoch': 14.285714285714286})

Share to huggingface

In [ ]:
model.push_to_hub("DarkMidoriya49/bloom-1b1-lora-tagger",
                  use_auth_token=True,
                  commit_message="Basic Training",
                  private=True)

/usr/local/lib/python3.11/dist-packages/transformers/utils/hub.py:922: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/DarkMidoriya49/bloom-1b1-lora-tagger/commit/b8ea5ad77b3bb994b886b1df67489937167505c6', commit_message='Basic Training', commit_description='', oid='b8ea5ad77b3bb994b886b1df67489937167505c6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/DarkMidoriya49/bloom-1b1-lora-tagger', endpoint='https://huggingface.co', repo_type='model', repo_id='DarkMidoriya49/bloom-1b1-lora-tagger'), pr_revision=None, pr_num=None)

Load Adapters From the Hub

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM

peft_model_id = "DarkMidoriya49/bloom-1b1-lora-tagger"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model = PeftModel.from_pretrained(model,peft_model_id)

adapter_config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


adapter_model.safetensors:   0%|          | 0.00/9.44M [00:00<?, ?B/s]

In [ ]:
model

(PeftModelForCausalLM(
   (base_model): LoraModel(
     (model): BloomForCausalLM(
       (transformer): BloomModel(
         (word_embeddings): Embedding(250880, 1536)
         (word_embeddings_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
         (h): ModuleList(
           (0-23): 24 x BloomBlock(
             (input_layernorm): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
             (self_attention): BloomAttention(
               (query_key_value): lora.Linear8bitLt(
                 (base_layer): Linear8bitLt(in_features=1536, out_features=4608, bias=True)
                 (lora_dropout): ModuleDict(
                   (default): Identity()
                 )
                 (lora_A): ModuleDict(
                   (default): Linear(in_features=1536, out_features=16, bias=False)
                 )
                 (lora_B): ModuleDict(
                   (default): Linear(in_features=16, out_features=4608, bias=False)
                 )
        

Inference

In [ ]:
batch = tokenizer("Whats the size of multiplication of matrices of size 3*3 and 3*3?", return_tensors='pt')


with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0],skip_special_tokens=False))